# Image Detection Model Testing
This notebook will test image detection models against captured screenshots. I plan to validate the 3 models below:
1. https://universe.roboflow.com/d41/attack-shv5h
2. https://universe.roboflow.com/workspace-sgzwp/diablo4_test2
3. https://universe.roboflow.com/diablo-4-fojil/diablo-4

**Update**: to run these models, you must use the api's. Instead I will extract the datasets myself, and train the models using: 
```YOLO11m```


## Environment Setup

In [ ]:
%%capture
%pip install ultralytics

### Force Kernel to use CUDA Device 1

In [ ]:
import torch
import os

# Set the CUDA_VISIBLE_DEVICES environment variable to use GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of GPUs
    num_gpus = torch.cuda.device_count()
    print(f'Number of GPUs: {num_gpus}')

    # List each GPU's name and other details
    for i in range(num_gpus):
        print(f'GPU {i}: {torch.cuda.get_device_name(i)}')
else:
    print('CUDA is not available.')

# Validate the CUDA_VISIBLE_DEVICES environment variable
print(f"CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES')}")


## Training

In [ ]:
import os
from ultralytics import YOLO

model = YOLO('yolo11m.pt')  # This downloads and loads the YOLO11m pretrained weights on device 1

# Specify the path to the data configuration file (data.yaml)
data_path = os.path.expanduser('~/Pictures/Diablo4.v2a.yolov11/data.yaml')

# Train the model
# Customize the epochs, batch size, and other parameters as needed
model.train(
    data=data_path,       # Path to the data.yaml file
    epochs=50,            # Number of epochs to train
    imgsz=640,            # Image size (adjust as needed)
    batch=12,              # Batch size (adjust based on GPU memory)
    workers=4,            # Number of data-loading workers
)

# Create the export directory if it doesn't exist
export_dir = os.path.expanduser('~/export')
os.makedirs(export_dir, exist_ok=True)  # Create the directory if it doesn’t exist

# Save the model weights after training
model.save(os.path.join(export_dir, 'yolo11m_trained.pt'))

## Inference
Configure image outputs to be rendered in the notebook.

In [ ]:
%matplotlib inline

In [10]:
import os
import yaml
from ultralytics import YOLO
import cv2

# This should be defined in the training cell, but in case you are just running inference
data_path = os.path.expanduser('~/Pictures/Diablo4.v2a.yolov11/data.yaml')

# Load the trained YOLO model
model_path = os.path.expanduser('~/export/yolo11m_trained.pt')
model = YOLO(model_path)

# Specify the paths
image_folder = os.path.expanduser('~/Pictures/Old D4 Captures/')  # Input images folder
output_folder = os.path.expanduser('~/Pictures/Old D4 Captures/results')  # Folder for annotated images
labels_folder = os.path.join(output_folder, "labels")  # Folder for YOLO TXT labels
os.makedirs(output_folder, exist_ok=True)  # Create the output directory if it doesn't exist
os.makedirs(labels_folder, exist_ok=True)  # Create the labels directory

# Load class names from data.yaml
with open(data_path, 'r') as f:
    data = yaml.safe_load(f)

# Create classes.txt file
classes_file_path = os.path.join(output_folder, 'classes.txt')
with open(classes_file_path, 'w') as classes_file:
    for class_name in data['names']:
        classes_file.write(f"{class_name}\n")

# Run inference on each image in the specified directory
for image_filename in os.listdir(image_folder):
    if image_filename.endswith(('.jpg', '.jpeg', '.png')):  # Adjust according to your image types
        # Load the image
        image_path = os.path.join(image_folder, image_filename)
        image = cv2.imread(image_path)
        
        # Run inference
        results = model.predict(image)

        # Process and visualize results
        label_file_path = os.path.join(labels_folder, f"{os.path.splitext(image_filename)[0]}.txt")
        with open(label_file_path, 'w') as f:
            for result in results:
                # Draw bounding boxes on the image and write labels to file
                boxes = result.boxes  # Get the bounding boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
                    conf = box.conf[0]           # Confidence score
                    label = int(box.cls[0])      # Class label
                    
                    # Calculate YOLO format bounding box coordinates
                    img_height, img_width = image.shape[:2]
                    x_center = (x1 + x2) / 2 / img_width
                    y_center = (y1 + y2) / 2 / img_height
                    width = (x2 - x1) / img_width
                    height = (y2 - y1) / img_height
                    
                    # Write to label file in YOLO format
                    f.write(f"{label} {x_center} {y_center} {width} {height}\n")

        # Save the output image with bounding boxes
        output_img_path = os.path.join(output_folder, image_filename)
        cv2.imwrite(output_img_path, image)